In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf_stock
from utils import broker_fee, get_date, round_half_up, get_asset_value
from fetch_data import fetch_exchange_rate
import requests
from secret import OXCR_KEY
from datetime import date, timedelta, datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
a = input("a: ")

try:
    a = int(a)
    if a not in [1, 2]:
        raise ValueError
except ValueError:
    if a != "m":
        print("\nI dati inseriti non sono corretti.")
        input("\nPremi Invio per tornare al Menu Principale...")
        raise KeyboardInterrupt
    
print(a)

1
